## Converting data format form apolloScape to Kaggle

In [ ]:
import numpy as np
import glob
import os
import cv2
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import json
import matplotlib.pylab as plt
import numpy as np
import glob
import os
import cv2
import pandas as pd
import seaborn as sns
from tqdm import tqdm

%matplotlib inline

%load_ext autoreload
%autoreload 2

In [ ]:
# https://raw.githubusercontent.com/ApolloScapeAuto/dataset-api/master/car_instance/car_models.py
from car_models import car_name2id, car_id2name
from utils import euler_to_rot, draw_obj, get_avg_size, get_intrinsics, read_json, write_json


## load image and render mask

In [ ]:
basedir = '/Users/pliu/Downloads/pku_kaggle_data/'
image_dir = '/Users/pliu/Downloads/pku_kaggle_data/train_images'
model_dir = '/Users/pliu/Downloads/pku_kaggle_data/car_models_json'

In [ ]:
from car_renderer import Visualizer
vis = Visualizer(basedir=basedir, image_dir=image_dir, model_dir=model_dir, save_dir=None)
len(vis.model_files)

In [ ]:
# Load an image
img_name = 'ID_001d6829a' 
# img_name = '180116_054054500_Camera_5' 
img = vis.load_image(img_name)

In [ ]:
# plt.figure(figsize=(20, 20))
# plt.imshow(img);

In [ ]:
overlay, bboxes = vis.process(img_name)

In [ ]:
plt.figure(figsize=(20, 10))
plt.imshow(np.hstack([img, overlay[...]]))

## apolloScape renderer api



In [ ]:
import sys
import argparse
apollo_api_dir = '/Users/pliu/github/6DVNET'
apollo_api_dir2 = '/Users/pliu/github/6DVNET/tools/ApolloScape_car_instance/'
sys.path.append(apollo_api_dir)
sys.path.append(apollo_api_dir2)

from tools.ApolloScape_car_instance.render_car_mesh import CarPoseVisualizer
from collections import namedtuple

Args = namedtuple('Args', [
    'image_name'        , # The name of a car type
    'data_dir'          , # id for specific car type
    'save_dir'    , # The name of the car category, 'SUV', 'Sedan' etc
    ])

In [ ]:
data_dir = '/Users/pliu/Downloads/3d_car_instance_sample/'
save_dir = '/Users/pliu/Downloads/apollo_scape_vis/'
image_name = '180116_054054500_Camera_5' #'all'
args = Args(image_name=image_name, data_dir=data_dir, save_dir=save_dir)

In [ ]:
settings = 'test'
print('Test visualizer')
visualizer = CarPoseVisualizer(args)
visualizer.load_car_models()
if args.image_name.lower() == 'all':
    image_name_list = [x[:-4] for x in os.listdir(visualizer._data_config['image_dir'])]
else:
    image_name_list = [args.image_name]
for image_name in tqdm(image_name_list):
    visualizer.showAnn(image_name, settings=settings, save_dir=args.save_dir)


## Rotation matrix representation


In [ ]:
rotmat = np.array([[ 0.99885411,  0.02962827,  0.03758506],
 [ 0.03545985, -0.9855819,  -0.16544157],
 [ 0.03214141,  0.16658475, -0.98550315]])

In [ ]:
from scipy.spatial.transform import Rotation as R
r = R.from_dcm(rotmat)

In [ ]:
# rotational matrix
r.as_dcm()

In [ ]:
np.isclose(np.dot(rotmat.T, rotmat), np.eye(3))

In [ ]:
r.as_quat()

In [ ]:
r.as_rotvec()

In [ ]:
r.as_euler('zxy', degrees=True)

`zyx` mean first applying z, then y, then x. In other words, `r_x * r_y * r_z`

In [ ]:
r = R.from_euler('zyx', [[90, 45, 30]], degrees=True)
r.as_euler('zyx', degrees=True)

In [ ]:
r_z = R.from_euler('z', 90, degrees=True)
r_y = R.from_euler('y', 45, degrees=True)
r_x = R.from_euler('x', 30, degrees=True)
(r_x * r_y * r_z).as_euler('zyx', degrees=True)

## apolloScape rotation

In [ ]:
from apollo_to_kaggle import euler_to_rot, euler_angles_to_rotation_matrix

In [ ]:
r = R.from_dcm(euler_angles_to_rotation_matrix([-8, 21.45, 30], degrees=True))

In [ ]:
r.as_euler('xyz', degrees=True)

## Kaggle rotation

In [ ]:
r = R.from_dcm(euler_to_rot(-8, 21.45, 30, degrees=True))

In [ ]:
r.as_euler('yxz', degrees=True)

Note that the kaggle and apolloScape angle decomposition is different. Kaggle uses a weird decoposition `yxz`

## conversion from apollo to kaggle

In [ ]:
from apollo_to_kaggle import convert_apollo_angle_to_kaggle

The following expriment explains when yaw is small, angles are good. However when yaw is large, the angles are off

In [ ]:
angle = [-4, 21.45, 30]
yaw, pitch, roll = convert_apollo_angle_to_kaggle(angle, degrees=True)
kaggle_angle = pitch, yaw, roll  # pitch and yaw are flipped
print('apollo {} --> kaggle {}'.format(angle, kaggle_angle))

In [ ]:
angle = [87, 21.45, 30]
yaw, pitch, roll = convert_apollo_angle_to_kaggle(angle, degrees=True)
kaggle_angle = pitch, yaw, roll  # pitch and yaw are flipped
print('apollo {} --> kaggle {}'.format(angle, kaggle_angle))

In [ ]:
# use radian
angle = [-1, 0.215, 0.30]
yaw, pitch, roll = convert_apollo_angle_to_kaggle(angle, degrees=False)
kaggle_angle = pitch, yaw, roll  # pitch and yaw are flipped
print('apollo {} --> kaggle {}'.format(angle, kaggle_angle))

In [ ]:
from apollo_to_kaggle import generate_kaggle_train_csv

In [ ]:
## batch conversion
data_dir = '/Users/pliu/Downloads/3d_car_instance_sample/'
pose_dir = f'{data_dir}/car_poses'
image_dir = f'{data_dir}/images'
save_path = f'{data_dir}/train.csv'

generate_kaggle_train_csv(pose_dir, image_dir, save_path)

## sanity check generate train.csv

In [ ]:
basedir = '/Users/pliu/Downloads/3d_car_instance_sample/'
image_dir = '/Users/pliu/Downloads/3d_car_instance_sample/images/'
model_dir = '/Users/pliu/Downloads/3d_car_instance_sample/car_models_json'

In [ ]:
from car_renderer import Visualizer
vis = Visualizer(basedir=basedir, image_dir=image_dir, model_dir=model_dir, save_dir=None)
len(vis.model_files)

In [ ]:
# Load an image
img_name = '180116_054054500_Camera_5' 
img = vis.load_image(img_name)

In [ ]:
# plt.figure(figsize=(20, 20))
# plt.imshow(img);

In [ ]:
overlay, bboxes = vis.process(img_name)